In [37]:
import simpy
import random
import numpy as np
from collections import OrderedDict

totalTime = 0 # Stores total waiting time in the process
avgSum = 0 # Stores the sum of the averages of waiting time in the simulations
accWait = 0 # Stores the sum of the time between stoping th arrivals and ending the procces
lastClient = None # Las client of this simulation
accMax = 0 # Stores the sum of the maximun waiting time in all the process in the simulation
maxDuration = 0 # Maximun duration seen in this simulation

def NonHomogenousExponential(Intensities, TotalTime, CurrTime):
     cand = random.expovariate()
     if(TotalTime < CurrTime):
          return cand
     for item, value in intensities:
          if(item >= CurrTime/TotalTime):
               if(random.uniform(0 , 1) < CurrTime/TotalTime):
                    return cand
               return cand + NonHomogenousExponential(Intensities, TotalTime, CurrTime + cand )
def client(env, name, servers, dist_servers, serviceTime):
    t = env.now
   # print(f'{name} entered in the system at t={t:.2f}')
    
    for i in range(len(servers)):
        with servers[i].request() as req:
            yield req # wait for the server to accept me
         #   print(f'{name} started service in server {i+1} at t={env.now:.2f}')
            yield env.timeout(dist_servers[i]()) # wait until the server finishs with me
         #   print(f'{name} finished service in server {i+1} at t={env.now:.2f}')

    # Finished with all n servers
    #print(f'{name} left the system at t={env.now:.2f}')
    global totalTime
    global maxDuration
    global lastClient
    global accMax
    totalTime += (env.now - t) 
    if(env.now - t > maxDuration):
            maxDuration = env.now - t
    if(name == lastClient):
        global accWait
        accWait += (env.now - serviceTime) 
        accMax += (maxDuration) ** 2
        maxDuration = 0
    
def setup(env, num_servers, dist_arrival, dist_servers, time):
    # Setups the simulation and creates the client´s process
    servers = [simpy.Resource(env) for _ in range(num_servers)]
    
    # Create clients
    i = 0
    while (env.now < time ):
        yield env.timeout(dist_arrival(env.now, time)) # wait until new client arrives
        newClient = client(env, f'Client {i}', servers, dist_servers, time)
        env.process(newClient) # process new client
        i += 1

    global lastClient
    lastClient = f'Client {i - 1}'
    global avgSum
    avgSum += totalTime/i

# Create environment


# Set number of servers, and time distributions to use in each one

    
dist_servers = [ lambda: random.expovariate(1.0 / 4),
                 lambda: random.expovariate(1.0 / 2), 
                 lambda: random.expovariate(1.0 / 3)]


for i in range(1):
    lastClient = None
    totalTime = 0
    env = simpy.Environment()
        # Setup the environment, and set the time distribution for client arrival
    intensities = OrderedDict([(0.5 , 1.0 / 5) , (1 , 1.0 / 6)])
    env.process(setup(env, len(dist_servers), lambda a, t: NonHomogenousExponential(intensities, a, t), dist_servers, 500))
    env.run()  # Run the simulation for a period of time 
print('el promedio es')
print(avgSum/1000)
print(accWait/1000)
print(accMax/1000)


el promedio es
12.042335434921352
2968.7508503002905
2968.055890905983
